In [1]:
import sympy as smp
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from symfit import parameters, variables, sin, cos, Fit
import scipy as sp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
def nextpow2(x):
    pow = 2
    while True:
        if pow <= x:
            pow = pow*2
        else:
            return pow
            break


In [94]:
N = 200
Fs = 4*N
Spacing = int(N/2)/5


x = np.full(int(N/2), 5)
x = np.append(x, np.linspace(5-5/int(N/2), 0, int(N/2)))

y = np.linspace(0, 5, int(N/2))
y = np.append(y, np.full(int(N/2), 5))

a = 1

Num_Turns = 10

df = pd.DataFrame(x, columns=['x'], dtype='float')
df['y'] = y
df['x'] = x
df['z'] = np.linspace(0, a, num=N)

df['comp'] = df['x'] + df['y']*1j
df['r'] = np.absolute(df['comp'])
df['theta'] = np.angle(df['comp'])

blank_df = df.copy()

for i in range(Num_Turns*4 - 1):
    new_df = blank_df.copy()
    new_df['z'] = new_df['z'] + (i+1)*a
    new_df['theta'] = new_df['theta'] + (i+1)*np.pi/2
    df = df.append(new_df)
    

df = df.reset_index(drop=True)

df['x'] = df['r']*np.cos(df['theta'])
df['y'] = df['r']*np.sin(df['theta'])

In [95]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=df['x'], y=df['y'], z=df['z'], mode='lines'))

In [96]:
# returns un-normalised gradient array of a n-dimension array.
# array must be in form [[x1, y1, z1, ...], [x2, y2, z2, ...], ...]
def grad(vector_array, normalise=False):
    count = 1
    grad_array = []
    
    for i in vector_array:
        if count == 1:
            last_i = i
            count += 1    
            
        else:
            grad = i - last_i
            grad_array.append(grad)
            count += 1
            last_i = i
    
    grad_array = grad_array[0:1] + grad_array
    
    grad_array = np.stack(grad_array)
    
    if normalise:
        norm_grad_list = []
        
        for vector in grad_array:
            norm_vector = vector/np.linalg.norm(vector)
            norm_grad_list.append(norm_vector)
        
        grad_array = np.stack(norm_grad_list)
            
    return grad_array

In [93]:
    def map_points(self, unmapped_local_array, normal_array, trans_array):
        """
        will map point from global origin (0, 0, 0) to translated and rotated points
        :param unmapped_local_array: n-dimensional array of points to be mapped.
        :param normal_array: n-dimensional array of normals to rotation point.
        :param trans_array: n-dimensional array of translated origins.
        :return: mapped_array: n-dimensional array of mapped points.
        """

        origin_normal = np.array([0, 1, 0])
        points_df = pd.DataFrame(normal_array, columns=['normal'])
        points_df['unmapped'] = unmapped_local_array
        points_df['translation'] = trans_array
        points_df['mapped'] = np.array(np.zeros(unmapped_local_array.shape()))

        for index, row in points_df.iterrows():
            normal_vector = row['normal']
            c = np.dot(origin_normal, normal_vector)
            axis = np.unitcross(origin_normal, normal_vector)
            s = np.sqrt(1 - c**2)
            C = 1 - c
            x = axis[0]
            y = axis[1]
            z = axis[2]
            rot_matrix = np.array([[ x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                                   [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
                                   [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])

            points_df['mapped'][index] = np.dot(rot_matrix, row['unmapped']) + row['translation']

        return pd.array(points_df['mapped'])

In [92]:
vector_array = np.transpose(np.array([np.array(df['x']), np.array(df['y']), np.array(df['z'])]))
grad_array = grad(vector_array, normalise=True)

<ipython-input-88-d2a4f2bef9a4>:26: RuntimeWarning:

invalid value encountered in true_divide



array([[0.00000000e+00, 9.95542448e-01, 9.43145477e-02],
       [0.00000000e+00, 9.95542448e-01, 9.43145477e-02],
       [0.00000000e+00, 9.95542448e-01, 9.43145477e-02],
       ...,
       [0.00000000e+00, 9.94505453e-01, 1.04684785e-01],
       [3.53319313e-15, 9.94505453e-01, 1.04684785e-01],
       [1.76659656e-15, 9.94505453e-01, 1.04684785e-01]])

In [91]:
N = 200

i = np.full(int(N/2), 5)
i = np.append(x, np.linspace(5-5/int(N/2), 0, int(N/2)))

j = np.linspace(0, 5, int(N/2))
j = np.append(y, np.full(int(N/2), 5))

a = 1

Num_Turns = 10

df = pd.DataFrame(x, columns=['x'], dtype='float')
df['y'] = y
df['x'] = x
df['z'] = np.linspace(0, a, num=N)

df['comp'] = df['x'] + df['y']*1j
df['r'] = np.absolute(df['comp'])
df['theta'] = np.angle(df['comp'])

blank_df = df.copy()

for i in range(Num_Turns*4 - 1):
    new_df = blank_df.copy()
    new_df['z'] = new_df['z'] + (i+1)*a
    new_df['theta'] = new_df['theta'] + (i+1)*np.pi/2
    df = df.append(new_df)
    

df = df.reset_index(drop=True)

df['x'] = df['r']*np.cos(df['theta'])
df['y'] = df['r']*np.sin(df['theta'])

(800, 3)

In [101]:
major_length = 25
minor_length = 10
revolution_length = 2
n_per_revolution = 100

perimeter = 2*major_length + 2*minor_length

x = np.full(int(n_per_revolution*(minor_length/2)/perimeter), major_length/2)

x = np.append(x, np.linspace(major_length/2, -major_length/2,
                             int(n_per_revolution*(major_length)/perimeter)))
              
x = np.append(x, np.full(int(n_per_revolution*(minor_length)/perimeter), -major_length/2))
              
x = np.append(x, np.linspace(-major_length/2, major_length/2,
                             int(n_per_revolution*(major_length)/perimeter)))
              
x = np.append(x, np.full(int(n_per_revolution*(minor_length/2)/perimeter), major_length/2))


              
y = np.linspace(0, minor_length/2, int(n_per_revolution*(minor_length/2)/perimeter))

y = np.append(y, np.full(int(n_per_revolution*(major_length)/perimeter), minor_length/2))

y = np.append(y, np.linspace(minor_length/2, -minor_length/2, int(n_per_revolution*minor_length/perimeter)))

y = np.append(y, np.full(int(n_per_revolution*major_length/perimeter), -minor_length/2))


              

array([ 12.5       ,  12.5       ,  12.5       ,  12.5       ,
        12.5       ,  12.5       ,  12.5       ,  12.5       ,
        11.76470588,  11.02941176,  10.29411765,   9.55882353,
         8.82352941,   8.08823529,   7.35294118,   6.61764706,
         5.88235294,   5.14705882,   4.41176471,   3.67647059,
         2.94117647,   2.20588235,   1.47058824,   0.73529412,
         0.        ,  -0.73529412,  -1.47058824,  -2.20588235,
        -2.94117647,  -3.67647059,  -4.41176471,  -5.14705882,
        -5.88235294,  -6.61764706,  -7.35294118,  -8.08823529,
        -8.82352941,  -9.55882353, -10.29411765, -11.02941176,
       -11.76470588, -12.5       , -12.5       , -12.5       ,
       -12.5       , -12.5       , -12.5       , -12.5       ,
       -12.5       , -12.5       , -12.5       , -12.5       ,
       -12.5       , -12.5       , -12.5       , -12.5       ,
       -12.5       , -11.76470588, -11.02941176, -10.29411765,
        -9.55882353,  -8.82352941,  -8.08823529,  -7.35

In [176]:
def make_rectangle(width, height, length, n):
    corners = [(width/2, 0),
              (width/2, height/2),
              (-width/2, height/2),
              (-width/2, -height/2),
              (width/2, -height/2),
              (width/2, 0)]
    x_list = []
    y_list = []
    perimeter = 2*width + 2*height
    
    def magnitude(p1, p2):
        p_diff = np.array([p1[0] - p2[0], p1[1] - p2[1]])
        mag = (p_diff[0]**2 + p_diff[1]**2)**(0.5)
        return mag
    
    for i in range(1, len(corners)):
        corner = corners[i]
        last_corner = corners[i-1]
        
        if i == 1:
            x_points = np.linspace(last_corner[0], corner[0], int(magnitude(corner, last_corner)*n/perimeter))
            x_list.append(x_points)
            y_points = np.linspace(last_corner[1], corner[1], int(magnitude(corner, last_corner)*n/perimeter))
            y_list.append(y_points)
            
        if i == len(corners):
            x_points = np.linspace(last_corner[0], corner[0], int(round(magnitude(corner, last_corner))*n/perimeter))
            x_list.append(x_points)
            y_points = np.linspace(last_corner[1], corner[1], int(round(magnitude(corner, last_corner))*n/perimeter))
            y_list.append(y_points)
            
        else:
            x_points = np.linspace(last_corner[0] - int(magnitude(corner, last_corner)*n/perimeter),
                                   corner[0],
                                   int(magnitude(corner, last_corner)*n/perimeter))
            x_list.append(x_points)
            y_points = np.linspace(last_corner[1],
                                   corner[1],
                                   int(magnitude(corner, last_corner)*n/perimeter))
            y_list.append(y_points)
        
    x = np.concatenate(x_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    z = np.linspace(0, length, y.shape[0])
    
    return x, y, z
        
x, y, z = make_rectangle(10, 30, 2, 100)

In [174]:
def make_helix(single_spiral, no_of_turns, output=False):
    helix_list = [[], [], []]
    length = single_spiral[2][-1]
    for i in range(no_of_turns):
        nth_spiral = np.copy(single_spiral)
        nth_spiral[2] = single_spiral[2] + i*length
        for i in range(3):
            helix_list[i].append(nth_spiral[i])
        helix_array = np.array([np.concatenate(helix_list[0]),
                                np.concatenate(helix_list[1]),
                                np.concatenate(helix_list[2])])
    if output:
        return helix_array
    
s_spiral = np.array([x, y, z])

b_helix = make_helix(s_spiral, 5, output=True)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=b_helix[0], y=b_helix[1], z=b_helix[2], mode='lines'))

In [171]:
grad_array = grad(np.transpose(b_helix))
points_array = map_points()

array([[0.        , 0.88235294, 0.02083333],
       [0.        , 0.88235294, 0.02083333],
       [0.        , 0.88235294, 0.02083333],
       ...,
       [0.        , 0.88235294, 0.02083333],
       [0.        , 0.88235294, 0.02083333],
       [0.        , 0.88235294, 0.02083333]])

In [ ]:
Length = nextpow2(df.shape[0])
df['fft_x'] = np.fft.fft(df['x'])
df['fft_y'] = np.fft.fft(df['y'])
df['fft_z'] = np.fft.fft(df['z'])

freq = np.linspace(0.0, Fs/(2.0*Spacing), int(df.shape[0]/2))

fig = go.Figure()

fig.add_trace(go.Scatter(x=freq, y=np.absolute(df['fft_x'][:int(df.shape[0]/2)]), mode='markers'))

fig.add_annotation(x=2, y=5,
            text="Text annotation with arrow",
            showarrow=True,
            arrowhead=1)

In [172]:
def map_points(unmapped_local_array, normal_array, trans_array):
    """
    will map point from global origin [0, 0, 0] to translated and rotated points
    :param unmapped_local_array: n-dimensional array of points to be mapped.
    :param normal_array: n-dimensional array of normals to rotation point.
    :param trans_array: n-dimensional array of translated origins.
    :return: mapped_array: n-dimensional array of mapped points.
    """

    origin_normal = np.array([0, 1, 0])
    points_df = pd.DataFrame(normal_array, columns=['normal'])
    points_df['unmapped'] = unmapped_local_array
    points_df['translation'] = trans_array
    points_df['mapped'] = np.array(np.zeros(unmapped_local_array.shape()))

    for index, row in points_df.iterrows():
        normal_vector = row['normal']
        c = np.dot(origin_normal, normal_vector)
        axis = np.unitcross(origin_normal, normal_vector)
        s = np.sqrt(1 - c**2)
        C = 1 - c
        x = axis[0]
        y = axis[1]
        z = axis[2]
        rot_matrix = np.array([[x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                               [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
                               [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])

        points_df['mapped'][index] = np.dot(rot_matrix, row['unmapped']) + row['translation']

    return pd.array(points_df['mapped'])

3